In [ ]:
import re
import tweepy
from tweepy import OAuthHandler
from textblob import TextBlob
from tkinter import *
import pandas as pd

  
class TwitterClient(object):
  
	def __init__(self):

		consumer_key = 'LmQw9UtVj9vauPEYrFzB6IU69'
		consumer_secret = 'Dn0Iv1lIk6bxDvNT2clnJiHSiq0mTcgUSX41HjaVthqzaGeAjo'
		access_token = '1475135994986860544-L5SfQtGQDS3qHxG6lm5QBKGciccG0A'
		access_token_secret = 'jKgNJBwNfb5HnFlr5SAA6jJgwk1usKwC7dQAj8NDkiruT'

		try:
			self.auth = OAuthHandler(consumer_key, consumer_secret)
			self.auth.set_access_token(access_token, access_token_secret)
			self.api = tweepy.API(self.auth)
		except:
			print("Error: Authentication Failed")

	def clean_tweet(self, tweet):
		return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", text).split())

	def get_tweet_sentiment(self, tweet):
		analysis = TextBlob(self.clean_tweet(tweet))
		if analysis.sentiment.polarity > 0.2:
			return 'positive'
		elif analysis.sentiment.polarity == 0:
			return 'neutral'
		else:
			return 'negative'

	def get_tweets_api(self, query, count):
		tweets = []
		try:
			tweets_accquired = self.api.search_tweets(q = query, count = count)
			for tweet in tweets_accquired:
				parsed_tweet = {}
				parsed_tweet['text'] = tweet.text
				parsed_tweet['sentiment'] = self.get_tweet_sentiment(tweet.text)
				if tweet.retweet_count > 0:
					if parsed_tweet not in tweets:
						tweets.append(parsed_tweet)
				else:
					tweets.append(parsed_tweet)
			return tweets
      
		except tweepy.errors.TweepyException as e:
			print("Error : " + str(e))

	def get_tweets_dataset(self, count):
		tweets_list = []
		data = pd.read_csv('Twitter Product Sentiment Analysis.csv')
		i = 0
		for tweet in data['text']:
			if i == count:
				break
			parsed_tweet = {}
			parsed_tweet['text'] = tweet
			parsed_tweet['sentiment'] = self.get_tweet_sentiment(tweet)
			tweets_list.append(parsed_tweet)
		return tweets_list
      

def twit():
  api = TwitterClient()
  phrase = phrase_entry.get()
  if len(phrase) != 0:
    tweets = api.get_tweets_api(query = phrase, count = 200)
  else:
    tweets = api.get_tweets_dataset(count = 20)
  ptweets = [tweet for tweet in tweets if tweet['sentiment'] == 'positive']
  print("Positive tweets percentage: {} %".format(100*len(ptweets)/len(tweets)))
  ntweets = [tweet for tweet in tweets if tweet['sentiment'] == 'negative']
  print("Negative tweets percentage: {} %".format(100*len(ntweets)/len(tweets)))
  print("Neutral tweets percentage: {} % ".format(100*(len(tweets)-(len( ntweets )+len( ptweets)))/len(tweets)))
  print("\n\nPositive tweets:")
  for tweet in ptweets[:10]:
    print(tweet['text']+'\n')
  print("\n\nNegative tweets:")
  for tweet in ntweets[:10]:
    print(tweet['text']+'\n')
  print('---------------------------------------------------------------------------')


  
if __name__ == "__main__":
  window = Tk()
  window.geometry('350x100')
  window.title('Phrase Input')
  window.config(padx=10, pady=10, bg='#FFCCCC')
  canvas = Canvas(height=200, width=200, highlightthickness=0, bg='#fdffb6')
  
  phrase_label = Label(text='Enter the phrase here:')
  phrase_label.grid(column=1, row=1,sticky='w')
  
  phrase_entry = Entry(window,width=40)
  phrase_entry.grid(column=1, row=3)
  phrase_entry.focus()
  
  search_button = Button(window,
    text='Find', highlightthickness=0, bg='light blue',fg="black", width=14, command=twit)
  search_button.grid(column=1, row=4)

  window.mainloop()
